In [12]:
import requests
import json
from typing import Tuple,List
import time
import datetime
import random
from login import Login
import pandas as pd
import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/ 537.36 Edg/107.0.1418.62'
}

In [4]:
with open('following.json', 'r') as f:
    following_list = json.loads(f.read())
len(following_list)

830

In [5]:
def compare_following(old:list,new:list):
    add = set(new).difference(set(old))
    unfollow = set(old).difference(set(new))
    return {'add':list(add),
            'unfollow':list(unfollow)}

In [6]:
def get_uid_list(following_list):
    ups = [i['mid'] for i in following_list]
    return ups

In [10]:
def get_mid(session: requests.Session) -> int:
    res = session.get(
        url='https://api.bilibili.com/nav', headers=headers)
    res = json.loads(res.text)
    data = res['data']
    return data['mid']

In [13]:
# Get the total number of followings

def get_total_follow(session: requests.Session) -> int:
    mid = get_mid(session)
    res = session.get(url='https://api.bilibili.com/x/relation/followings',
                      headers=headers,
                      params={'vmid': mid})
    data = json.loads(res.text)['data']
    return data['total']

# Get following list


def get_follow_list(session: requests.Session, page: int = 1) -> List[dict]:
    mid = get_mid(session)
    res = session.get(url='https://api.bilibili.com/x/relation/followings',
                      headers=headers,
                      params={'vmid': mid, 'pn': page, 'ps': 50})
    data = json.loads(res.text)['data']
    return data['list']

In [8]:
login = Login()
url, token = login.apply_QR_code()
login.generate_QR_code(url)
input('已扫码')
bilibili_session = requests.Session()
code = login.do_web_login(bilibili_session, token)
if code == 0:
    print('登录成功')
elif code == 86038:
    print('二维码失效')
elif code == 86090 or code == 86101:
    print('请扫码并确认')
else:
    print(f'错误，code:{code}')

█████████████████████████████████████████████████████
█████████████████████████████████████████████████████
████ ▄▄▄▄▄ ███▀▀▄▀█▀▄▄▀ █▄█▀▀▄▀▄█ ▄  ██▀▄█ ▄▄▄▄▄ ████
████ █   █ █ ▀▄▀▄ ▄▀▄█▀█▀▀▀▀▄▄▀▄▄▄ ▀▄▄█ ▀█ █   █ ████
████ █▄▄▄█ █ ▄▀█▀ ▄   █▄ ▄▄▄  ▀▀▀█▄█▄▀▄▄▄█ █▄▄▄█ ████
████▄▄▄▄▄▄▄█ █ █▄▀▄█ █▄▀ █▄█ ▀ ▀ ▀ █ ▀▄█ █▄▄▄▄▄▄▄████
████ █▄▄▄ ▄▀▀█▄▀     ▄▀  ▄▄▄ ▀███   ▀▄█▀▀█▄▄   ▀▀████
████ █▄█▄ ▄ ▀ █ ██ ▄█ ▀▄█ ▄▀ ▀    ▀▀  ▄▄▀▄▄▄▀▄ ▄█████
████▀█▀▄██▄▄▄▄██▄█▀▀ █▄▄██ ▄██▄▀▀█ ▄▀█▄ ▀██▄▀▀▀▀ ████
██████ ▀ █▄ █▀█▀ ▄██ ▄▄▀▄▄ ▄ █▄▀▀▄▀█▀█ ▄ ▀▄ ▀▀ ██████
████  ▄▀ ▄▄ █▀ ▄ ██▄▄▄▄▄█ ▀██▄█▀█▀▀▄██▄ ▀▀▄▄██▀▄▀████
████▄ █▄ █▄ ▄ ▀ █▀██ ▄ ▀▄▀█ ▄▄▄ ▀ █▀▀ ▄ █ ▄▄▀   █████
████▄ ▀  ▄▄▄  ██▀▀▀█▀▀▄▀ ▄▄▄ ▀▄█▀ ▀▀███  ▄▄▄ ▀▀▀ ████
████ ▀▀▄ █▄█ ▀█ ▀▀█ █▀   █▄█ █ ▀▀██▄▀ ▄█ █▄█ ▀ ██████
████ ▀ ▀ ▄▄▄▄▄▄███▄▀█▀ ▀   ▄  █▀▀▄  ▀██▀ ▄ ▄▄ ██ ████
███████▄▄▀▄▀███▀▀█▀█▄  ▀▄▀ ▀▀▀  ▀▄▀  ▀▄▀▄▀▀ █  ▄█████
████▄▀ ▀▀ ▄▄▄ ▄▀ ███▀█ █▀▄▀▄▄ ▄ ▀▄  ▀█▄▀ █ █▄▀▀ ▀████
████ ▀█▄██▄▀▀▀█▄▄▄ ██ ▀ ▀▄▄▄▀▄▄▄▀ ▀  █▄▀▄█▀ █▄ ██████
████▄█▄█▄▄▄▀▀▀█ █ ▀ ▄▄█ ▄▄▀ 

In [14]:
total = get_total_follow(bilibili_session)
print(total)
following_list_new = []
for page in range(1, total // 51 + 2):
    time.sleep(random.randint(1, 3))
    ups = get_follow_list(bilibili_session, page)
    print(page, len(ups))
    following_list_new.extend(ups)
print(len(following_list_new))

850
1 50
2 50
3 50
4 50
5 50
6 50
7 50
8 50
9 50
10 50
11 50
12 50
13 50
14 50
15 50
16 50
17 50
850


In [17]:
diff = compare_following(get_uid_list(following_list),get_uid_list(following_list_new))
diff['add']

[690112648,
 516413836,
 478904588,
 1314329237,
 38421915,
 690545051,
 294081438,
 595298729,
 46508980,
 3461582812088629,
 286187082,
 12995787,
 1607984338,
 410904791,
 477745885,
 310362975,
 241378147,
 1787553002,
 417264380,
 9048063]